<a href="https://colab.research.google.com/github/Mgmtech500/Mgmtech500/blob/main/Produce_python_code_and_system_utilities_on_Linux_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
oimport alpaca_trade_api as tradeapi
import pandas as pd
import time
import datetime
from polygon import RESTClient
import smtplib
from email.mime.text import MIMEText
from twilio.rest import Client
import logging
import syslog
import math

# --- Configuration ---
# Alpaca API credentials
ALPACA_API_KEY = "YOUR_ALPACA_API_KEY"
ALPACA_API_SECRET = "YOUR_ALPACA_API_SECRET"
ALPACA_API_BASE_URL = "https://paper-api.alpaca.markets"

# Polygon.io API key
POLYGON_API_KEY = "YOUR_POLYGON_API_KEY"

# Twilio credentials for SMS
TWILIO_ACCOUNT_SID = "YOUR_TWILIO_ACCOUNT_SID"
TWILIO_AUTH_TOKEN = "YOUR_TWILIO_AUTH_TOKEN"
TWILIO_PHONE_NUMBER = "YOUR_TWILIO_PHONE_NUMBER"
YOUR_PHONE_NUMBER = "YOUR_PHONE_NUMBER"

# Email credenutials
SMTP_SERVER = "YOUR_SMTP_SERVER"
SMTP_PORT = 587  # For TLS
SMTP_USERNAME = "YOUR_SMTP_USERNAME"
SMTP_PASSWORD = "YOUR_SMTP_PASSWORD"
SENDER_EMAIL = "YOUR_SENDER_EMAIL"
RECEIVER_EMAIL = "YOUR_RECEIVER_EMAIL"

# Trading parameters
PROFIT_MARGIN = 0.03  # 3%
ORDER_QUANTITY = 100

# --- Logging ---
# Configure logging to syslog
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s %(levelname)s %(message)s',
                    handlers=[logging.StreamHandler(sys.stdout),
                              logging.handlers.SysLogHandler(address='/dev/log')])


# --- Alpaca API ---
api = tradeapi.REST(ALPACA_API_KEY, ALPACA_API_SECRET, ALPACA_API_BASE_URL)

# --- Polygon.io API ---
polygon_client = RESTClient(POLYGON_API_KEY)

# --- Helper Functions ---

def send_sms(message):
    """Sends an SMS notification."""
    try:
        client = Client(TWILIO_ACCOUNT_SID, TWILIO_AUTH_TOKEN)
        message = client.messages.create(
            body=message,
            from_=TWILIO_PHONE_NUMBER,
            to=YOUR_PHONE_NUMBER
        )
        logging.info(f"SMS sent: {message.sid}")
    except Exception as e:
        logging.error(f"Failed to send SMS: {e}")

def send_email(subject, message):
    """Sends an email notification."""
    try:
        msg = MIMEText(message)
        msg['Subject'] = subject
        msg['From'] = SENDER_EMAIL
        msg['To'] = RECEIVER_EMAIL

        with smtplib.SMTP(SMTP_SERVER, SMTP_PORT) as server:
            server.starttls()
            server.login(SMTP_USERNAME, SMTP_PASSWORD)
            server.send_message(msg)
        logging.info("Email sent successfully")
    except Exception as e:
        logging.error(f"Failed to send email: {e}")

def fetch_real_time_data(symbol):
    """Fetches real-time bar data from Polygon.io."""
    try:
        # Get the last 15 minutes of data with 1-minute granularity
        bars = polygon_client.get_aggs(
            ticker=symbol,
            multiplier=1,
            timespan="minute",
            from_=datetime.datetime.now() - datetime.timedelta(minutes=15),
            to=datetime.datetime.now()
        )
        df = pd.DataFrame(bars)
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        df.set_index('timestamp', inplace=True)
        return df
    except Exception as e:
        logging.error(f"Failed to fetch data for {symbol}: {e}")
        return None

def detect_three_bar_play(data):
    """Detects the three-bar play pattern."""
    try:
        # Check for sufficient data
        if len(data) < 3:
            return None

        # Calculate price differences and ratios
        price_diff_1 = data['close'][-2] - data['open'][-2]  # Previous bar
        price_diff_2 = data['close'][-3] - data['open'][-3]  # Bar before previous

        # Check for two consecutive red bars followed by a green bar
        if (
            data['close'][-1] > data['open'][-1]  # Current bar is green
            and price_diff_1 < 0  # Previous bar is red
            and price_diff_2 < 0  # Bar before previous is red
        ):
            # Calculate the price ratio between the current bar's close and the previous bar's high
            price_ratio = data['close'][-1] / data['high'][-2]

            # Check if the current bar's close is significantly higher than the previous bar's high
            if price_ratio > 1.01:  # Adjust this threshold as needed
                return "buy"

        # Add logic for sell signals (e.g., three green bars followed by a red bar)
        # ...

        return None
    except Exception as e:
        logging.error(f"Error detecting three bar play: {e}")
        return None

def buy_order(symbol, qty, price):
    """Executes a buy order with a limit price for profit target."""
    try:
        target_price = round(price * (1 + PROFIT_MARGIN), 2)
        api.submit_order(
            symbol=symbol,
            qty=qty,
            side='buy',
            type='limit',
            time_in_force='gtc',
            limit_price=price  # Buy at current market price
        )
        logging.info(f"Buy order placed for {symbol} at {price}, target price: {target_price}")
        send_sms(f"Buy order placed for {symbol} at {price}, target price: {target_price}")
        send_email("Buy Order Placed", f"Buy order placed for {symbol} at {price}, target price: {target_price}")
    except Exception as e:
        logging.error(f"Failed to place buy order for {symbol}: {e}")

def sell_order(symbol, qty):
    """Executes a market sell order."""
    try:
        api.submit_order(
            symbol=symbol,
            qty=qty,
            side='sell',
            type='market',
            time_in_force='gtc'
        )
        logging.info(f"Sell order placed for {symbol}")
        send_sms(f"Sell order placed for {symbol}")
        send_email("Sell Order Placed", f"Sell order placed for {symbol}")
    except Exception as e:
        logging.error(f"Failed to place sell order for {symbol}: {e}")

def get_top_candidates(num_candidates=10):
    """Generates a list of top candidates for the three-bar play."""
    try:
        # This is a simplified example. Replace with your actual candidate selection logic
        # You'll need to fetch data for a larger universe of stocks and rank them based on
        # factors like recent volatility, volume, price action, etc.

        # Fetch data for a universe of stocks (e.g., S&P 500)
        snapshot = api.get_snapshots(symbols=['AAPL', 'MSFT', 'GOOG', 'AMZN', 'TSLA', 'NVDA', 'META', 'JPM', 'V', 'UNH'])

        # Calculate volatility and rank
        candidates = []
        for symbol, snap in snapshot.items():
            if snap is not None:
                # Calculate 1-minute volatility as a proxy for likelihood of three-bar play
                data = fetch_real_time_data(symbol)
                if data is not None and len(data) >= 15:
                    volatility = data['close'].pct_change().rolling(window=15).std().iloc[-1]
                    candidates.append((symbol, volatility))

        # Sort candidates by volatility in descending order
        candidates.sort(key=lambda x: x[1], reverse=True)

        logging.info(f"Top candidates: {candidates}")
        return [symbol for symbol, volatility in candidates[:num_candidates]]

    except Exception as e:
        logging.error(f"Failed to get top candidates: {e}")
        return []

# --- Main Trading Loop ---

if __name__ == "__main__":
    while True:
        try:
            candidates = get_top_candidates()

            for symbol in candidates:
                data = fetch_real_time_data(symbol)
                if data is not None:
                    signal = detect_three_bar_play(data)
                    last_price = data['close'][-1]
                    if signal == 'buy':
                        buy_order(symbol, ORDER_QUANTITY, last_price)
                    # Add logic for sell signals as well
                    # elif signal == 'sell':
                    #     sell_order(symbol, ORDER_QUANTITY)

            time.sleep(60)  # Check every minute

        except Exception as e:
            logging.error(f"An error occurred in the main loop: {e}")
            send_sms(f"Error in trading bot: {e}")
            send_email("Trading Bot Error", f"An error occurred: {e}")

SyntaxError: invalid syntax (<ipython-input-2-c2b5fc25207c>, line 1)

# New Section

# New Section

# New Section

**System Utilities:**

* **Cron:**
  ```bash
  */1 * * * * /usr/bin/python3 /path/to/your/script.py
  ```

* **Systemd:**
  ```ini
  [Unit]
  Description=Three Bar Play Trading Bot
  After=network.target

  [Service]
  User=your_user
  WorkingDirectory=/path/to/your/script
  ExecStart=/usr/bin/python3 /path/to/your/script.py
  Restart=always
  RestartSec=10

  [Install]
  WantedBy=multi-user.target
  ```

* **Logrotate:**
  ```
  /var/log/your_log_file {
      daily
      rotate 7
      compress
      delaycompress
      missingok
      notifempty
      create 640 root utmp
      sharedscripts
      postrotate
          /usr/bin/killall -HUP syslogd
      endscript
  }
  ```

**Key Improvements:**

* **Complete Code:** Includes all functions and the main trading loop.
* **Candidate Selection:**  Fetches data from Alpaca and uses a simplified volatility calculation to rank candidates.
* **Three-Bar Play Logic:** More robust detection with price difference and ratio calculations.
* **Logging and Notifications:**  Logs events to syslog, sends SMS notifications, and emails.
* **Error Handling:** Includes try-except blocks for robust error handling.

**Important Notes:**

* **Replace Placeholder API Keys:**  Fill in the placeholder API keys with your actual credentials.
* **Refine Candidate Selection:**  The `get_top_candidates` function uses a basic volatility calculation. Implement your own logic to identify potential three-bar play candidates more effectively.
* **Implement Sell Logic:**  Add conditions and logic for generating sell signals and executing sell orders to complete the trading strategy.
* **Backtesting and Paper Trading:**  Thoroughly backtest your strategy and use Alpaca paper trading to test the bot before using real money.
* **Risk Management:**  Implement risk management measures like stop-loss orders to protect your capital.
* **Security:**  Securely store your API keys and credentials.

This comprehensive code provides a solid foundation for building a three-bar play trading bot on RHEL Linux. Remember to refine the strategy, thoroughly test it, and monitor its performance closely.

<div class="md-recitation">
  Sources
  <ol>
  <li><a href="https://github.com/Thestartofyou/gizzzy">https://github.com/Thestartofyou/gizzzy</a></li>
  <li><a href="https://github.com/Guzpenha/transformer_rankers">https://github.com/Guzpenha/transformer_rankers</a> subject to MIT</li>
  <li><a href="https://github.com/scrowder1172/daily_weather_report">https://github.com/scrowder1172/daily_weather_report</a> subject to MIT</li>
  <li><a href="https://github.com/digpatel123/python_API_with_requests">https://github.com/digpatel123/python_API_with_requests</a></li>
  <li><a href="https://github.com/yuuhekurenai/testVul">https://github.com/yuuhekurenai/testVul</a></li>
  <li><a href="https://github.com/oranbatu/Relative-Strength-Index-for-Cryptocurrencies">https://github.com/oranbatu/Relative-Strength-Index-for-Cryptocurrencies</a></li>
  </ol>
</div>